In [18]:
import pandas as pd
import os
# import sklearn

path = "/workspaces/codespaces-jupyter/data/YAAD/"
stimulus_descripttion_file = pd.read_excel(path + 'Stimulus_Description.xlsx')

fear_video_id = stimulus_descripttion_file.query('`Target Emotion` == "fear"').head(1).get('Video ID').item()

path_single_modal_data = path + "RawData/Singlemodal/ECG/"
path_multi_modal_data = path + "RawData/Multimodal/ECG/"

single_modal_data_files = os.listdir(path_single_modal_data)
multi_modal_data_files = os.listdir(path_multi_modal_data)

single_modal_dfs = []
for file in single_modal_data_files:
    if "v" + str(fear_video_id) in file:
        df = pd.read_csv(path_single_modal_data + file, delimiter='\t')
        single_modal_dfs.append(df)

multi_modal_dfs = []
for file in multi_modal_data_files:
    if "v" + str(fear_video_id) in file:
        df = pd.read_csv(path_multi_modal_data + file, delimiter='\t')
        multi_modal_dfs.append(df)

In [19]:
def str_dfs_to_real_dfs(dfs) -> pd.array:
    real_dfs = []
    for df in dfs:
        arr = df.columns.values
        str_vals = arr[0].split(',')
        real_vals = [float(each) for each in str_vals]
        real_vals_pd = pd.array(real_vals)
        real_dfs.append(real_vals_pd)
    return pd.array(real_dfs)

def mean_of_means(dfs) -> float:
    t = []
    for each in dfs:
        t.append(each.mean())
    return pd.array(t).mean()

def min_of_mins(dfs) -> float:
    t = []
    for each in dfs:
        t.append(each.min())
    return pd.array(t).min()

def max_of_maxs(dfs) -> float:
    t = []
    for each in dfs:
        t.append(each.max())
    return pd.array(t).max()

def mean_of_vars(dfs) -> float:
    t = []
    for each in dfs:
        t.append(each.var())
    return pd.array(t).mean()

In [20]:
real_sm_dfs = str_dfs_to_real_dfs(single_modal_dfs)
real_mm_dfs = str_dfs_to_real_dfs(multi_modal_dfs)

single refers to singlemodal data
multi refers to multimodal data

In [21]:
print("mean of means single: ", mean_of_means(real_sm_dfs))
print("mean of means multi: ", mean_of_means(real_mm_dfs))

print("min of mins single: ", min_of_mins(real_sm_dfs))
print("min of mins multi: ", min_of_mins(real_mm_dfs))

print("max of maxs single: ", max_of_maxs(real_sm_dfs))
print("max of maxs multi: ", max_of_maxs(real_mm_dfs))

print("mean of vars single: ", mean_of_vars(real_sm_dfs))
print("mean of vars multi: ", mean_of_vars(real_mm_dfs))

mean of means single:  -18.522971550372002
mean of means multi:  -10.029048553400726
min of mins single:  -297.21
min of mins multi:  -21.106
max of maxs single:  18.678
max of maxs multi:  8.1532
mean of vars single:  0.13076550723324526
mean of vars multi:  0.041651835869248255
